In [24]:
import time
import functools
import math
import random

In [25]:
# basic timer (use timeit module for more percision)
def b1_timer(fun) :
    "print runtime of decorated function"
    @functools.wraps(fun)
    def t_wrapper(*args, **kwargs) :
        start_t = time.perf_counter()
        r_val = fun(*args, **kwargs)
        end_t = time.perf_counter()
        run_t = end_t - start_t
        print(f"{fun.__name__}()  time:{run_t:0.4f} seconds") # or {fun.__name__!r} == repr()
        return r_val
    return t_wrapper

@b1_timer
def b1_spin(repz, idx):
    for _ in range(repz):
        sum([i**2 for i in range(99999)])
    return idx+1
b1r = 44
b1i = 1
print(f"{b1_spin(b1r, b1i)}")

b1_spin()  time:0.4372 seconds
2


In [26]:
# a useful logger
def b2_debug(fun):
    """print a signature + return value for logging calls"""
    @functools.wraps(fun)
    def d_wrapper(*args, **kwargs) :
        rep_args = [repr(a) for a in args]
        rep_kwargs = [f"{k}={v!r}" for k, v in kwargs.items()]
        sig = ", ".join(rep_args + rep_kwargs)
        print(f"{fun.__name__}({sig})")
        rv = fun(*args, **kwargs)
        print(f"-->  {rv}\n")
        return rv
    return d_wrapper

@b2_debug
def b2_test(name, idx=None):
    if idx is None :
        print(f" failed, name={name}")
        return -1
    print(f"name={name}  ,  idx={idx}")
    return idx+1
b2_test(name="potatoe")
b2_test("sweet potatoe", 9)
b2_test(name="sweet potatoe", idx=5)

b2_test(name='potatoe')
 failed, name=potatoe
-->  -1

b2_test('sweet potatoe', 9)
name=sweet potatoe  ,  idx=9
-->  10

b2_test(name='sweet potatoe', idx=5)
name=sweet potatoe  ,  idx=5
-->  6



6

In [27]:
# apply a decorator to std library function
math.factorial = b2_debug(math.factorial)
def approx_e(terms=4) :
    "e = sum(0, oo, 1/n!)"
    return sum((1/math.factorial(n)) for n in range(terms))
approx_e(5)

factorial(0)
factorial(0)
-->  1

-->  1

factorial(1)
factorial(1)
-->  1

-->  1

factorial(2)
factorial(2)
-->  2

-->  2

factorial(3)
factorial(3)
-->  6

-->  6

factorial(4)
factorial(4)
-->  24

-->  24



2.708333333333333

In [28]:
# slow down, rate limit (ie only check every second if a server is available)
def b3_slow(fun) :
    """sleep 1 s before call"""
    @functools.wraps(fun)
    def s_wrap(*args, **kwargs) :
        time.sleep(1)
        return fun(*args, **kwargs)
    return s_wrap

@b3_slow
def b3_test(n_from):
    if n_from < 1 :
        print("complete")
    else :
        print(n_from)
        b3_test(n_from-1) # recursion
b3_test(5)

5
4
3
2
1
complete


In [41]:
# decorator not used on function, just confirms function exists and returns unwrapped
b4_PLUGINS = dict()

def b4_register(fun) :
    """registers the function as a plugin"""
    b4_PLUGINS[fun.__name__] = fun
    return fun # this is the original function, unmodified (no inner wrap needed)

@b4_register
def b4_imbelish(name):
    return f"{name} is a good person"

@b4_register
def b4_slander(name):
    return f"{name} is a terrorist"

def b4_judge(name):
    k,v = random.choice(list(b4_PLUGINS.items()))
    return v(name)
print(f"{b4_judge('Mike')} | {b4_judge('51-cent')} | {b4_judge('Vladimir')}")
# just define a function and decorate it with register to keep track of your list
# this effictively picks your functions from globals()


Mike is a terrorist | 51-cent is a terrorist | Vladimir is a terrorist
